In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb

train_df = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
x_train_src, x_test_scr = train_test_split(train_df, test_size=0.5, random_state=0)

x_train = x_train_src.drop('claim', axis=1)
y_train = x_train_src.iloc[:,-1]
result_df = pd.DataFrame()

params = {
    'n_estimators': 100,
    'random_state': 0
}

cv = KFold(n_splits=5, random_state=0, shuffle=True)

model = lgb.LGBMClassifier(**params)
result_df['initial'] = cross_val_score(model, x_train, y_train, cv=cv)

model = make_pipeline(
    StandardScaler(),
    lgb.LGBMClassifier(**params)
)
result_df['StandardScaler'] = cross_val_score(model, x_train, y_train, cv=cv)

model = make_pipeline(
    MinMaxScaler(),
    lgb.LGBMClassifier(**params)
)
result_df['MinMaxScaler'] = cross_val_score(model, x_train, y_train, cv=cv)

model = make_pipeline(
    RobustScaler(),
    lgb.LGBMClassifier(**params)
)
result_df['RobustScaler'] = cross_val_score(model, x_train, y_train, cv=cv)

model = lgb.LGBMClassifier(**params)
result_df['log1p'] = cross_val_score(model, np.log1p(x_train), y_train, cv=cv)

display(result_df.T)